In [ ]:

import cv2
from ultralytics import YOLO
import easyocr
import os
import re


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
trained_model_path = r"E:\internship project\runs\detect\train3\weights\best.pt"
model = YOLO(trained_model_path)

print("✅ YOLO model loaded from:", trained_model_path)


✅ YOLO model loaded from: E:\internship project\runs\detect\train3\weights\best.pt


In [3]:
reader = easyocr.Reader(['en'])
print("✅ EasyOCR reader initialized")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete✅ EasyOCR reader initialized


In [6]:
image_path = r"E:\internship project\Detection Model\detection_dataset\images\val\241.jpg"

print("✅ Using image:", image_path)


✅ Using image: E:\internship project\Detection Model\detection_dataset\images\val\241.jpg


In [7]:
results = model(image_path)
image = cv2.imread(image_path)



image 1/1 E:\internship project\Detection Model\detection_dataset\images\val\241.jpg: 224x640 1 name, 2 uids, 1 address, 64.3ms
Speed: 5.9ms preprocess, 64.3ms inference, 10.6ms postprocess per image at shape (1, 3, 224, 640)


In [8]:
extracted_data = {}

for result in results[0].boxes.data.tolist():
    x1, y1, x2, y2, confidence, class_id = map(int, result[:6])
    field_class = model.names[class_id]   # "name", "uid", "address"
    
    # crop region
    cropped = image[y1:y2, x1:x2]
    
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    text = reader.readtext(gray, detail=0)
    extracted_text = " ".join(text).strip()

    extracted_data[field_class] = extracted_text

extracted_data


e:\internship project\env\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'uid': '5415 6207 8561',
 'address': 'Address: 1064 Second Near Boom Plaza Sector 57 , on, Gurgaon; Haryana 1528a8 Floor,',
 'name': 'Salil Ahuja'}

In [9]:
validated = {}

# ✅ UID validation (12 digits)
uid = extracted_data.get("uid", "")
uid_clean = re.sub(r"\D", "", uid)
validated["uid_valid"] = len(uid_clean) == 12
validated["uid_value"] = uid_clean

# ✅ Name validation
name = extracted_data.get("name", "")
validated["name_valid"] = bool(re.match(r"^[A-Za-z ]{3,}$", name))
validated["name_value"] = name

# ✅ Address validation
address = extracted_data.get("address", "")
validated["address_valid"] = len(address) > 10
validated["address_value"] = address

validated


{'uid_valid': True,
 'uid_value': '541562078561',
 'name_valid': True,
 'name_value': 'Salil Ahuja',
 'address_valid': True,
 'address_value': 'Address: 1064 Second Near Boom Plaza Sector 57 , on, Gurgaon; Haryana 1528a8 Floor,'}

In [10]:
print("\n✅ FINAL EXTRACTED DATA")
for k, v in extracted_data.items():
    print(f"{k.upper()} : {v}")

print("\n✅ VALIDATION RESULTS")
for k, v in validated.items():
    print(f"{k} : {v}")



✅ FINAL EXTRACTED DATA
UID : 5415 6207 8561
ADDRESS : Address: 1064 Second Near Boom Plaza Sector 57 , on, Gurgaon; Haryana 1528a8 Floor,
NAME : Salil Ahuja

✅ VALIDATION RESULTS
uid_valid : True
uid_value : 541562078561
name_valid : True
name_value : Salil Ahuja
address_valid : True
address_value : Address: 1064 Second Near Boom Plaza Sector 57 , on, Gurgaon; Haryana 1528a8 Floor,
